# Sentiment analysis with Logistic Regression

## Logistic Regression

> Some of the explanations in this tutoarial are inspired by chapters 3 and 8 in the book _"Python Machine Learning"_ by Sebastian Raschka

First, let's introduce what Logistic Regression is. 

[Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) is a classification model that is very easy to implement and performs very well on linearly separable classes. It is one of the most widely used
algorithms for classification in industry too, which makes it attractive to play with.

_Very_ simplistically explained, Logistic Regression works as follows:

![Logistic Regression](./images/logistic_regression.png)

First we will define the input for our algorithm. The imput will be each sample in whatever dataset we are working with. Each sample will consist of several features. For example, if we're working with housing price prediction, the features for each sample could be the size of the house, number of rooms, etc. We'll call the input vector **X**.

For the algorythm to learn, we need to define variables that we can adjust accordingly to what we want to predict. We will create a vector of _weights_ (**W**) that the model will adjust in order to predict more accurately. The process of adjusting those weights is what we call **learning**.

For every input sample, we will perform a dot product of the features by the weights **XW**. This product is sometimes referred as _net input_. This will give us a real number. Since in this particular problem we want to _classify_ (positive/negative), we need squash this number in the range [0, 1]. This will give us the _probability_ of a positive event. A function that does precisely that is called **sigmoid**. The sigmoid function looks like this:

![sigmoid](./images/sigmoid.svg)

What sigmoid is doing is basically transforming big inputs into a value close to 1, and small inputs into a value close to 0. This is exactly what we want. 

We will do this for every sample in our training set and compute the errors. To calculate the error we only need to compare our prediction with the true label for each sample. We will sum the square errors of all the samples to get a global prediction error. This will be our **cost function**.

A cost function is then something we want to minimize. **Gradient descent** is a method for finding the minimum of a function of multiple variables, such like the one we're dealing with here.

### Gradient descent

> Watch [this](https://www.youtube.com/watch?v=IHZwWFHWa-w) video for a visual introduction to Gradient Descent

Once all our training samples have been computed and the error calculated with our cost function, we need to _minimize_ that cost function. A method for doing that is gradient descent. There are many [articles](https://en.wikipedia.org/wiki/Gradient_descent) that contain detailed explanations _and_ implementations of GD, so let's not do this here. However is good to have an intuition.

For illustration purposes, let's think about a function with two parameters. Something like this one:

![Gradient Descent](./images/Gradient_descent.png)

Gradient descent will try to find the minimum of the function. To do so, we calculate the slope of the function at a certain point, and move towards the direction that makes the function decrease. There are some things to have in mind though.

As you can see a function can have one or several _local minimum_. In a local minimum, the slope will be zero and GD will "think" it's found the global minimum. To avoid this, we can choose a bigger "step" when we move towards the minimum. The "size" of the step towards the minimun is what we call the **learning rate**, and it's another adjustable parameter.

We need to be careful here: If we choose a too small learning rate, we can get stuck in a local minimum. If we instead choose a too big learning rate, we risk overshooting the global minimum. We need to experiment, and the adecuate learning rate depends on the particular problem and the data.

### The training process

In order for logistic regression to learn, we need to repeat the process descrived before several times. Each one of these times is called an **epoch**. The number of epochs to run depends on the problem and the training data. It is... yes, another tunnable parameter of the algorithm.

The set of all tunnable parameters is called **hyperparameters** of the model.

Like with the leatning rate, we need to be careful when choosing the number of epochs: If we train too many epochs, we risk **overfitting**. This means that our model will "memorize" the training data and will generalize badly when presented new data. 

If we train too little, it will fail to find any pattern and the prediction accuracy will be very low. This is known as **underfitting**.

There are techniques that help prevent overfitting. These **regularization** techniques are out of the scope of this tutorial, but... guess! It's also something to tune and experiment with :)

This is why when training a model you need to set aside a _test dataset_ in order to know the accuracy of your algorithm in unknown data. The test dataset will **never** be used during training

## Sentiment analysis

Let's first of all have a look at the data

In [8]:
import pandas as pd

#Load the data into a DataFrame
train = pd.read_csv('data/train.csv', encoding='latin-1')
test = pd.read_csv('data/test.csv', encoding='latin-1')

train.head(10)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
5,6,0,or i just worry too much?
6,7,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
7,8,0,Sunny Again Work Tomorrow :-| ...
8,9,1,handed in my uniform today . i miss you ...
9,10,1,hmmmm.... i wonder how she my number @-)


As we can see, the structure of a twit varies a lot between twit and twit. They have different lengths, letters, numbers, extrange characters, etc. 

It is also important to note that **a lot** of words are not correctly spelled, for example the word _"Juuuuuuuuuuuuuuuuussssst"_

This makes it hard to mesure how positive or negative are the words withing the corpus of twits. If they were all correct dictionary words, we could use a **lexicon** to punctuate words. However because of the nature of social media language, we cannot do that. 

So we need a way of scoring the words such that words that appear in positive twits have greater score that those that appear in negative twits.

But first... how do we represent the twits as vectors we can input to our algorithm?

### Bag of words

One thing we could do to represent the twits as equal-sized vectors of numbers is the following:

* Create a list (vocabulary) with all the unique words in the whole corpus of twits. 
* We construct a feature vector from each twit that contains the counts of how often each word occurs in the particular twit

_Note that since the unique words in each twit represent only a small subset of all the words in the bag-of-words vocabulary, the feature vectors will mostly consist of zeros_

Let's construct the bag of words. We will work with a smaller example for illustrative purposes, and at the end we will work with our real data.

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

twits = [
    'This is amazing!',
    'ML is the best',
    'I am not sure about how this is going to end...'
]

count = CountVectorizer()
bag = count.fit_transform(twits)

count.vocabulary_

{'about': 0,
 'am': 1,
 'amazing': 2,
 'best': 3,
 'end': 4,
 'going': 5,
 'how': 6,
 'is': 7,
 'ml': 8,
 'not': 9,
 'sure': 10,
 'the': 11,
 'this': 12,
 'to': 13}

As we can see from executing the preceding command, the vocabulary is stored in a Python dictionary that maps the unique words to integer indices. Next, let's print the feature vectors that we just created:

In [51]:
bag.toarray()

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1]], dtype=int64)

Each index position in the feature vectors corresponds to the integer values that are stored as dictionary items in the CountVectorizer vocabulary. For example, the first feature at index position 0 resembles the count of the word 'about' , which only occurs in the last document, and the word 'is' , at index position 7, occurs in all three twits. These values in the feature vectors are also called the raw term frequencies: `tf(t,d )` —the number of times a term `t` occurs in a document `d`.

### How relevant are words? Term frequency-inverse document frequency
